# Q1: Get data to the fin. agent
* Agents SDK OpenAI - create a few agents to read the fin. data

In [1]:
# 
!uv  pip list | grep openai

Using Python 3.12.7 environment at: /Users/realmistic/Documents/ai-bootcamp-codespace/.venv
openai                                   1.109.1
openai-agents                            0.3.3


In [2]:
import agents
print(f"📦 Version: {agents.__version__}")

📦 Version: 0.3.3


In [3]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = await Runner.run(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

A function calls self—  
Endless mirrors, shrinking down.  
Base case breaks the spell.


In [4]:
import requests
from agents import function_tool

In [5]:
# from agents-sdk file
def fetch_url(url):
    jina_reader_base_url = 'https://r.jina.ai/'
    jina_reader_url = jina_reader_base_url + url
    response = requests.get(jina_reader_url)
    return response.content.decode('utf-8')

In [6]:
from requests.exceptions import RequestException
from typing import Optional

# @function_tool
def fetch_url(url: str) -> Optional[str]:
    """
    Fetch the textual content of a webpage.

    Args:
        url (str): The target URL to fetch content from.

    Returns:
        Optional[str]: The decoded HTML/text content of the fetched page if successful,
        or None if an error occurred.

    Raises:
        ValueError: If the provided URL is empty or invalid.
    """
    if not url or not isinstance(url, str):
        raise ValueError("The 'url' parameter must be a non-empty string.")

    jina_reader_base_url = "https://r.jina.ai/"
    jina_reader_url = jina_reader_base_url + url.lstrip("/")

    try:
        response = requests.get(jina_reader_url, timeout=10)
        response.raise_for_status()  # Raises HTTPError for bad status codes
        return response.content.decode("utf-8")
    except RequestException as e:
        # Catch all network-related errors (e.g., ConnectionError, Timeout, HTTPError)
        print(f"Error fetching URL '{jina_reader_url}': {e}")
        return None
    except UnicodeDecodeError:
        print(f"Error decoding response from '{jina_reader_url}'.")
        return None


In [7]:
content = fetch_url('https://pythoninvest.com')

In [8]:
print(content)

Title: Use Your Computer to Make Informed Decisions in Stock Trading

URL Source: https://pythoninvest.com/

Published Time: Tue, 07 Oct 2025 21:34:40 GMT

Markdown Content:
Use Your Computer to Make Informed Decisions in Stock Trading


[To main content](https://pythoninvest.com/#t-main-content)

*   [Home](https://pythoninvest.com/)
*   [News Feed](https://pythoninvest.com/#weekly-fin-news-feed)
*   [Blog](https://pythoninvest.com/blog)
*   [Course](https://pythoninvest.com/course)
*   [Contacts](https://pythoninvest.com/contacts)

[Subscribe](https://pythoninvest.com/#popup:subscribe)

![Image 1](https://static.tildacdn.net/tild6364-3431-4735-b734-373031313037/Frame_2.svg)

![Image 2](https://static.tildacdn.net/tild3563-3230-4562-b337-326333623863/Frame_1.svg)

PYTHON

[I|]

[](https://pythoninvest.com/)

[**PYTHON**[|]](https://pythoninvest.com/)

*   [Home](https://pythoninvest.com/)
*   [News Feed](https://pythoninvest.com/#weekly-fin-news-feed)
*   [Blog](https://pythoninvest.c

In [9]:
web_agent = Agent(
    name='web_agent',
    instructions="you're a helful assistnat",
    model='gpt-4o-mini',
    tools=[function_tool(fetch_url)]
)

In [10]:
from agents import Runner

runner = Runner()

question = "what is this page about? pythoninvest.com"

In [11]:
results = await runner.run(web_agent, input=question)

In [12]:
print(results.final_output)

The page at **pythoninvest.com** is focused on helping individuals make informed decisions in stock trading using Python programming. It offers resources related to stock market trading, analytics, and data visualization. Key features include:

- **Courses:** Analytical courses that help users learn about stock trading using Python.
- **Blog:** Articles covering various aspects of financial analysis and trading strategies.
- **Weekly Analytical Review:** Insights and trends in the market.
- **Community Engagement:** Opportunities for users to join discussions and share feedback.

The site is geared towards analysts, developers, and students looking to enhance their skills in finance and programming.


In [13]:
results.new_items

[ToolCallItem(agent=Agent(name='web_agent', handoff_description=None, tools=[FunctionTool(name='fetch_url', description='Fetch the textual content of a webpage.', params_json_schema={'properties': {'url': {'description': 'The target URL to fetch content from.', 'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'fetch_url_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x110577a60>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)], mcp_servers=[], mcp_config={}, instructions="you're a helful assistnat", prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response

In [14]:
import yfinance as yf
import pandas as pd

In [15]:
# @function_tool
def get_eps_trend(ticker: str) -> dict:
    """
      Get the EPS (Earnings Per Share) trend for a given stock ticker.
      
      Args:
          ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')
      
      Returns:
          Dictionary containing EPS trend data for the specified ticker
      """
    ticker_obj = yf.Ticker(ticker)
    result = ticker_obj.get_eps_trend()
    # Convert DataFrame to dict if needed
    if isinstance(result, pd.DataFrame):
        return result.to_dict(orient='records')  # List of dicts
    return result
    

In [16]:
r = get_eps_trend("TSLA")
print(r)

[{'current': 0.43803, '7daysAgo': 0.44158, '30daysAgo': 0.46205, '60daysAgo': 0.46042, '90daysAgo': 0.4721}, {'current': 0.44611, '7daysAgo': 0.44842, '30daysAgo': 0.45767, '60daysAgo': 0.45152, '90daysAgo': 0.45152}, {'current': 1.65422, '7daysAgo': 1.69074, '30daysAgo': 1.68203, '60daysAgo': 1.6747, '90daysAgo': 1.67603}, {'current': 2.25553, '7daysAgo': 2.32696, '30daysAgo': 2.38618, '60daysAgo': 2.36581, '90daysAgo': 2.40581}]


In [17]:
# in case we need WebSearch 
from agents import WebSearchTool

In [18]:
stock_agent = Agent(
    name='stock_agent',
    instructions="you're a helful assistant. Try to use all defined tools before going into the WebSearchTool",
    model='gpt-4o-mini',
    tools=[ 
            # WebSearchTool(), 
           function_tool(get_eps_trend)] 
)

In [19]:
# results = await runner.run(stock_agent, 
#                            input="get me the eps trend for TSLA and the latest earnings call date and results, news sentiment")

In [20]:
print(results.final_output)

The page at **pythoninvest.com** is focused on helping individuals make informed decisions in stock trading using Python programming. It offers resources related to stock market trading, analytics, and data visualization. Key features include:

- **Courses:** Analytical courses that help users learn about stock trading using Python.
- **Blog:** Articles covering various aspects of financial analysis and trading strategies.
- **Weekly Analytical Review:** Insights and trends in the market.
- **Community Engagement:** Opportunities for users to join discussions and share feedback.

The site is geared towards analysts, developers, and students looking to enhance their skills in finance and programming.


In [21]:
from typing import Any
from datetime import datetime, timezone

def get_eps_trend(ticker: str) -> dict[str, Any]:
    """
    Get the EPS (Earnings Per Share) trend for a given stock ticker.

    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'GOOGL', 'MSFT')

    Returns:
        Dictionary with EPS trend data
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        result = ticker_obj.get_eps_trend()

        if isinstance(result, pd.DataFrame):
            if result.empty:
                return {"error": f"No EPS trend data available for {ticker}"}
            return {"ticker": ticker, "data": result.to_dict(orient='records')}
        return {"ticker": ticker, "data": result}
    except Exception as e:
        return {"error": f"Failed to get EPS trend for {ticker}: {str(e)}"}

def get_earnings_dates(ticker: str) -> dict[str, Any]:
    """
    Get earnings call dates for a stock ticker. The function all returns the expected EPS, the actual EPS, and the surprise percentage.
    It has stats from many quarters and years.
    It also returns the next earnings call date.
    Args:
        ticker: Stock ticker symbol

    Returns:
        Dictionary with earnings dates
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        result = ticker_obj.get_earnings_dates()

        # if dataframe - main case
        if isinstance(result, pd.DataFrame):
            if result.empty:
                return {"error": f"No earnings data available for {ticker}"}
        
            # Include the index (dates) as a column before converting to dict
            result = result.reset_index().rename(columns={"index": "date"})

            # return good stats
            return {"ticker": ticker, "data": result.to_dict(orient='records'), "date_of_run": str(datetime.now(timezone.utc).strftime("%Y-%m-%d"))}
        
        # return {"ticker": ticker, "data": result.to_dict(orient='records'), "date_of_run": str(datetime.now(timezone.utc).strftime("%Y-%m-%d"))}
        return {"error":"bad format - API returned not a dictionary"}
    except Exception as e:
        return {"error": f"Failed to get earnings dates for {ticker}: {str(e)}"}

In [22]:
r = get_earnings_dates("TSLA")
print(r)

{'ticker': 'TSLA', 'data': [{'Earnings Date': Timestamp('2026-01-28 15:00:00-0500', tz='America/New_York'), 'EPS Estimate': 0.44, 'Reported EPS': nan, 'Surprise(%)': nan}, {'Earnings Date': Timestamp('2025-10-22 16:00:00-0400', tz='America/New_York'), 'EPS Estimate': 0.56, 'Reported EPS': 0.5, 'Surprise(%)': -10.53}, {'Earnings Date': Timestamp('2025-07-23 16:00:00-0400', tz='America/New_York'), 'EPS Estimate': 0.4, 'Reported EPS': 0.4, 'Surprise(%)': -0.97}, {'Earnings Date': Timestamp('2025-04-22 16:00:00-0400', tz='America/New_York'), 'EPS Estimate': 0.41, 'Reported EPS': 0.27, 'Surprise(%)': -34.89}, {'Earnings Date': Timestamp('2025-01-29 16:00:00-0500', tz='America/New_York'), 'EPS Estimate': 0.77, 'Reported EPS': 0.73, 'Surprise(%)': -4.83}, {'Earnings Date': Timestamp('2024-10-23 16:00:00-0400', tz='America/New_York'), 'EPS Estimate': 0.5, 'Reported EPS': 0.62, 'Surprise(%)': 24.76}, {'Earnings Date': Timestamp('2024-07-23 20:00:00-0400', tz='America/New_York'), 'EPS Estimate':

In [23]:
# Create agent
stock_agent = Agent(
    name="Stock Analyst",
    instructions="""You are a stock market analyst. 
When tools return data, summarize it clearly for the user.
I need to understand the EPS growth trend and YoY, the surprise on average over the last years (CAGR) and last 4 quarters.
When is the next earnings call date (usually the date in the earnings calendar first/last, after than the current date)? 
If a tool returns an error, explain it and don't retry the same thing repeatedly.
Use Earnings Dates function first, as it contains more data.

If there is an error from an API - show the error to the user.""",
    tools=[
        # WebSearchTool(), 
        function_tool(get_eps_trend),
        function_tool(get_earnings_dates)
    ]
)


In [24]:
from IPython.display import display
from IPython.display import Markdown

display(Markdown("tada"))  # ensures full rendering

tada

In [25]:
# using display to ensure full output rendering in Jupyter

for ticker in ["META"]:
# ['AMZN', "AAPL", "GOOGL", "META", "MSFT"]:
    display(Markdown("---"*30))
    display(Markdown(f"--- Analyzing ticker: {ticker} ---"))

    results = await Runner.run(
        stock_agent,
        input=f"get me the eps trend for {ticker} and the latest earnings call date",
        max_turns=20  # Increase max turns
    )

    display(Markdown(results.final_output))
    display(Markdown("==="*30))

------------------------------------------------------------------------------------------

--- Analyzing ticker: META ---

Here’s a summary of the EPS (Earnings Per Share) growth trend, recent earnings surprises, and the next earnings call for META:

### EPS Growth Trend (Year-over-Year and Latest Quarters)
- EPS has shown strong growth over the recent quarters and years:
    - Last four quarters (actual EPS, newest to oldest): 7.25, 7.14, 6.43, 8.02
    - Prior year four quarters: 6.03, 5.16, 4.71, 5.33

- YoY comparison (using most recent comparable quarters):
    - EPS has grown significantly versus the prior year. For example: 
        - Q3: 7.25 (2025) vs. 6.03 (2024) → ~20% YoY growth
        - Q2: 7.14 (2025) vs. 5.16 (2024) → ~38% YoY growth
        - Q1: 6.43 (2025) vs. 4.71 (2024) → ~37% YoY growth
        - Q4: 8.02 (2025) vs. 5.33 (2024) → ~51% YoY growth

### EPS Surprise (Average)
- Last 4 quarters:
    - 8.66%, 21.82%, 23.53%, 19.00%
    - Average EPS surprise: ~18.75%
- Recent multi-year trend:
    - Last 3 years: EPS surprises have varied, but have averaged in the high single-digits to low double-digits, with some volatility (range from -20% to over +40%).

### Next Earnings Call
- The next scheduled earnings call for META is on January 28, 2026.

If you need historical CAGR calculations or more detail, let me know!

==========================================================================================

# Q2: Finance news db & chunking strategy

In [26]:
from datetime import datetime, timezone
datetime.now(timezone.utc).strftime("%Y-%m-%d")

'2025-10-31'

In [27]:
from datetime import datetime, timezone

# The exact time of running the Colab (in UTC, timezone-aware)
now_right_format = datetime.now(timezone.utc).isoformat(timespec='milliseconds')
print(now_right_format)

2025-10-31T22:41:35.182+00:00


In [28]:
from datetime import datetime, timezone

# The exact time of running the Colab
now = datetime.now(timezone.utc).isoformat(timespec='milliseconds').replace('+00:00', 'Z')  # RFC3339 format
now_right_format = now
print(now_right_format)

2025-10-31T22:41:35.188Z


In [33]:
import os
# check the API KEY is imported correctly from the .envrc file
print(len(os.getenv('POLYGON_API_KEY')))

POLYGON_API_KEY = os.getenv('POLYGON_API_KEY')

32


In [34]:
# https://polygon.io/docs/stocks/get_v2_reference_news
# https://polygon.io/blog/api-pagination-patterns/
# API CALL : # https://api.polygon.io/v2/reference/news?order=desc&limit=1000&sort=published_utc&apiKey=<your key> or POLYGON_API_KEY
      # need to get 200 OK status

# retrieve max 1000 news via one API call
def get_one_chunk_of_news_polygon_io(api_key = POLYGON_API_KEY, news_limit=1000, max_date = now_right_format):
  url = f"https://api.polygon.io/v2/reference/news?order=desc&limit={news_limit}&sort=published_utc&published_utc.lt={max_date}&apiKey={api_key}"

  # https://www.nylas.com/blog/use-python-requests-module-rest-apis/ - Python for rest APIs
  # try/catch for HTTP requests: https://stackoverflow.com/questions/16511337/correct-way-to-try-except-using-python-requests-module
  try:
      print(f'trying the API call : {url}')
      r = requests.get(url, timeout=3)
      r.raise_for_status()
  except requests.exceptions.HTTPError as errh:
      print ("Http Error:",errh)
  except requests.exceptions.ConnectionError as errc:
      print ("Error Connecting:",errc)
  except requests.exceptions.Timeout as errt:
      print ("Timeout Error:",errt)
  except requests.exceptions.RequestException as err:
      print ("OOps: Something Else",err)

  data = r.json()


  # Check if 'results' key exists in the response
  if 'results' in data:
      # If it exists, proceed with normalization
      df_nested_list = pd.json_normalize(data, record_path=['results'])
  else:
      # If not, print the response keys for debugging and create an empty DataFrame
      print(f"The 'results' key was not found in the response. Available keys are: {data.keys()}")
      df_nested_list = pd.DataFrame() # or handle it differently based on the new structure

  return df_nested_list

  # OLD CODE: # https://towardsdatascience.com/how-to-convert-json-into-a-pandas-dataframe-100b2ae1e0d8
  # df_nested_list = pd.json_normalize(data, record_path =['results'])
  # print(f'Retrieved : {len(df_nested_list)} news; min_datetime = {df_nested_list.published_utc.min()}, max_datetime = {df_nested_list.published_utc.max()}')
  # return df_nested_list

In [35]:
def get_all_news(api_calls_left = 5, api_key = POLYGON_API_KEY, news_limit=1000, max_date = now_right_format):
  all_news = None
  for i in range(api_calls_left):
    cur = get_one_chunk_of_news_polygon_io(api_key = api_key, news_limit = news_limit, max_date = max_date)
    if all_news is None:
      all_news = cur
    else:
      all_news = pd.concat([all_news,cur], ignore_index=True, axis=0) #stacking dataframes

    max_date = cur.published_utc.min() #update max_date of the news
  return all_news

In [36]:
# 5 calls per minute limit for a free account - all recent news (5000)
all_news = get_all_news()

trying the API call : https://api.polygon.io/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-10-31T22:41:35.188Z&apiKey=ChTGkHcZ52m1DGD_KCgVOUwbV3iJ61_X
trying the API call : https://api.polygon.io/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-10-27T20:00:00Z&apiKey=ChTGkHcZ52m1DGD_KCgVOUwbV3iJ61_X
trying the API call : https://api.polygon.io/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-10-22T13:00:00Z&apiKey=ChTGkHcZ52m1DGD_KCgVOUwbV3iJ61_X
trying the API call : https://api.polygon.io/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-10-16T15:14:04Z&apiKey=ChTGkHcZ52m1DGD_KCgVOUwbV3iJ61_X
trying the API call : https://api.polygon.io/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-10-10T21:33:22Z&apiKey=ChTGkHcZ52m1DGD_KCgVOUwbV3iJ61_X


In [38]:
all_news.head()

,id,title,author,published_utc,article_url,tickers,image_url,description,keywords,insights,publisher.name,publisher.homepage_url,publisher.logo_url,publisher.favicon_url,amp_url
0,fbd05842ba119aa51c71df671c546137b0c395b70d735a...,"JSPR DEADLINE: ROSEN, SKILLED INVESTOR COUNSEL...",Rosen Law Firm,2025-10-31T21:38:00Z,https://www.globenewswire.com/news-release/202...,"[JSPR, JSPRW]",https://ml.globenewswire.com/Resource/Download...,Rosen Law Firm is pursuing a securities class ...,"[securities lawsuit, class action, manufacturi...","[{'ticker': 'JSPR', 'sentiment': 'negative', '...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN
1,1b26ee6b521fb3390e0852ddb9053d910e58f56a6165af...,Capital Group Canada Announces Estimated 2025 ...,Prnewswire,2025-10-31T21:30:00Z,https://www.benzinga.com/pressreleases/25/10/n...,[CGCB],https://www.benzinga.com/next-assets/images/be...,Capital Group Canada released estimated annual...,"[ETFs, capital gains, distributions, tax year,...","[{'ticker': 'CGCB', 'sentiment': 'neutral', 's...",Benzinga,https://www.benzinga.com/,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN
2,c45000e9357bab55292e000cb8d97bf92de28278e5f1d6...,Caliber Launches Noteholder Debt-to-Equity Con...,Caliber Investor Relations,2025-10-31T21:25:00Z,https://www.globenewswire.com/news-release/202...,"[CWD, GLNK]",https://ml.globenewswire.com/Resource/Download...,Caliber announced a voluntary Noteholder Conve...,"[debt conversion, equity, financial restructur...","[{'ticker': 'CWD', 'sentiment': 'positive', 's...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN
3,f454b54fa7f7c606181104a3d8c08f9b0d626795c35daa...,Gran Tierra Announces Confirmation with Respec...,Globe Newswire,2025-10-31T21:21:00Z,https://www.benzinga.com/pressreleases/25/10/g...,[GTE],https://www.benzinga.com/next-assets/images/be...,Gran Tierra Energy confirmed compliance with p...,"[acquisition, takeover, scheme of arrangement,...","[{'ticker': 'GTE', 'sentiment': 'neutral', 'se...",Benzinga,https://www.benzinga.com/,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN
4,5bf11b738ec7881ca1e4bb34979062f1ed853d87eaa3ac...,Dynamix Corporation III Completes $201.25 Mill...,Globe Newswire,2025-10-31T21:13:50Z,https://www.benzinga.com/pressreleases/25/10/g...,"[COHN, ETHM, ETHMU, ETHMW]",https://www.benzinga.com/next-assets/images/be...,Dynamix Corporation III successfully completed...,"[IPO, SPAC, Nasdaq, energy infrastructure, dig...","[{'ticker': 'COHN', 'sentiment': 'neutral', 's...",Benzinga,https://www.benzinga.com/,https://s3.polygon.io/public/assets/news/logos...,https://s3.polygon.io/public/assets/news/favic...,NaN


In [52]:
sample_json = all_news.head(10).to_dict(orient='records')
# .to_json(orient='records', indent=2)
print(sample_json)

[{'id': 'fbd05842ba119aa51c71df671c546137b0c395b70d735a94724698973036bd88', 'title': 'JSPR DEADLINE: ROSEN, SKILLED INVESTOR COUNSEL, Encourages Jasper Therapeutics, Inc. Investors with Losses in Excess of $100K to Secure Counsel Before Important Deadline in Securities Class Action – JSPR', 'author': 'Rosen Law Firm', 'published_utc': '2025-10-31T21:38:00Z', 'article_url': 'https://www.globenewswire.com/news-release/2025/10/31/3178678/673/en/JSPR-DEADLINE-ROSEN-SKILLED-INVESTOR-COUNSEL-Encourages-Jasper-Therapeutics-Inc-Investors-with-Losses-in-Excess-of-100K-to-Secure-Counsel-Before-Important-Deadline-in-Securities-Cl.html', 'tickers': ['JSPR', 'JSPRW'], 'image_url': 'https://ml.globenewswire.com/Resource/Download/745335af-3a3a-4fb5-84c0-fdedc9adf092', 'description': 'Rosen Law Firm is pursuing a securities class action lawsuit against Therapeutics for allegedly making false and misleading statements about manufacturing controls and product prospects during the period of November 30, 

In [54]:
sample_json[0]

{'id': 'fbd05842ba119aa51c71df671c546137b0c395b70d735a94724698973036bd88',
 'title': 'JSPR DEADLINE: ROSEN, SKILLED INVESTOR COUNSEL, Encourages Jasper Therapeutics, Inc. Investors with Losses in Excess of $100K to Secure Counsel Before Important Deadline in Securities Class Action – JSPR',
 'author': 'Rosen Law Firm',
 'published_utc': '2025-10-31T21:38:00Z',
 'article_url': 'https://www.globenewswire.com/news-release/2025/10/31/3178678/673/en/JSPR-DEADLINE-ROSEN-SKILLED-INVESTOR-COUNSEL-Encourages-Jasper-Therapeutics-Inc-Investors-with-Losses-in-Excess-of-100K-to-Secure-Counsel-Before-Important-Deadline-in-Securities-Cl.html',
 'tickers': ['JSPR', 'JSPRW'],
 'image_url': 'https://ml.globenewswire.com/Resource/Download/745335af-3a3a-4fb5-84c0-fdedc9adf092',
 'description': 'Rosen Law Firm is pursuing a securities class action lawsuit against Therapeutics for allegedly making false and misleading statements about manufacturing controls and product prospects during the period of Novembe

In [55]:
all_news["publisher.name"].value_counts()

publisher.name
GlobeNewswire Inc.           2183
The Motley Fool              1493
Benzinga                     1021
Investing.com                 298
Zacks Investment Research       5
Name: count, dtype: int64

In [56]:
all_news_json = all_news.to_dict(orient='records')

In [58]:
len(all_news_json)

5000

In [ ]:
# Key features:

#   1. No chunking needed - Each news article is already a discrete document
#   2. Text search - Searches across title, description, keywords, author
#   3. Exact ticker matching - Function to find articles with specific ticker in tickers field
#   4. Multi-ticker support - Search for multiple tickers at once
#   5. Date filtering - Can filter news by publication date
#   6. Boosting - Title matches get higher priority than description

#   Note on chunking:
#   Unlike the podcast data (long markdown documents), news articles are already atomic units. Each article is a complete piece of information, so no chunking is
#   needed. The tickers field already provides precise ticker associations.


In [61]:

# Import minsearch
from minsearch import Index

# Step 1: Convert DataFrame to list of dicts (if not already done)
news_documents = all_news.to_dict(orient='records')

print(f"Total news articles: {len(news_documents)}")
print(f"Fields per article: {len(news_documents[0])}")
print(f"Sample article keys: {list(news_documents[0].keys())}")

Total news articles: 5000
Fields per article: 15
Sample article keys: ['id', 'title', 'author', 'published_utc', 'article_url', 'tickers', 'image_url', 'description', 'keywords', 'insights', 'publisher.name', 'publisher.homepage_url', 'publisher.logo_url', 'publisher.favicon_url', 'amp_url']


In [64]:
from tqdm import tqdm
  # Step 2: Preprocess documents - convert list fields to strings
print("\nPreprocessing documents...")
for doc in tqdm(news_documents, desc="Converting fields"):
    # Convert list fields to comma-separated strings
    if isinstance(doc.get('tickers'), list):
        doc['tickers'] = ', '.join(doc['tickers'])

    if isinstance(doc.get('keywords'), list):
        doc['keywords'] = ', '.join(doc['keywords'])

    # Ensure text fields are strings
    for field in ['title', 'description', 'author']:
        if doc.get(field) is None:
            doc[field] = ''
        elif not isinstance(doc.get(field), str):
            doc[field] = str(doc[field])

print("✓ Preprocessing complete")



Preprocessing documents...


Converting fields: 100%|████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 579836.32it/s]

✓ Preprocessing complete


In [65]:

  # Step 3: Create Index with relevant text fields
news_index = Index(
    text_fields=["title", "description", "keywords", "author", "tickers"],
    keyword_fields=["published_utc", "publisher.name"]
)

# Step 4: Fit the index with preprocessed documents
print("\nBuilding search index...")
news_index.fit(news_documents)
print(f"✓ Index created with {len(news_documents)} articles")




Building search index...
✓ Index created with 5000 articles


In [66]:
# Step 5: Search function to retrieve news for a ticker
def search_news_by_ticker(ticker, num_results=10):
    """
      Search for news articles related to a specific ticker.
      
      Args:
          ticker: Stock ticker symbol (e.g., 'TSLA', 'AAPL')
          num_results: Number of results to return
      
      Returns:
          List of news articles matching the ticker
    """
    results = news_index.search(
        query=ticker,
        num_results=num_results,
        boost_dict={
              'tickers': 5.0,      # Highest boost for ticker field
              'title': 3.0,         # High boost for title
              'description': 1.5,   # Medium boost for description
              'keywords': 1.0       # Standard boost for keywords
          }
      )
    return results

In [67]:

  # Step 6: Test - Get news for TSLA
print("\n" + "="*70)
print("Testing search for TSLA...")
print("="*70)

tsla_news = search_news_by_ticker("TSLA", num_results=5)

print(f"\nFound {len(tsla_news)} news articles for TSLA:\n")
for i, article in enumerate(tsla_news, 1):
    print(f"{i}. {article['title']}")
    print(f"   Published: {article['published_utc']}")
    print(f"   Tickers: {article['tickers']}")
    print(f"   Description: {article['description'][:100]}...")
    print()


Testing search for TSLA...

Found 5 news articles for TSLA:

1. Tesla Q3 Deliveries Smash Estimates, But Wall Street Wasn't Impressed. What Gives?
   Published: 2025-10-09T08:23:00Z
   Tickers: TSLA
   Description: Tesla reported nearly 497,100 vehicle deliveries in Q3, exceeding Wall Street estimates by 50,000 un...

2. Should You Buy Tesla Stock Before Wednesday Afternoon?
   Published: 2025-10-20T14:37:00Z
   Tickers: TSLA
   Description: Tesla prepares to report Q3 earnings amid declining revenue and challenges in the EV market, with an...

3. Tesla's New Robotaxi-Ready Models Strengthen the Bull Case for the Stock
   Published: 2025-10-09T07:10:00Z
   Tickers: TSLA
   Description: Tesla introduced new Standard trims for Model 3 and Model Y with prices under $40,000, featuring sel...

4. Tesla Finds New Buyer For Unsold Cybertrucks: SpaceX, xAI Stepping In
   Published: 2025-10-14T19:29:47Z
   Tickers: TSLA
   Description: Tesla is selling Cybertrucks to SpaceX and xAI due to low 

In [68]:
# Step 7: Get exact ticker matches (more precise)
def get_news_by_exact_ticker(ticker, news_docs=news_documents):
    """
    Get articles where ticker appears in the tickers field.
    More precise than text search.
    """
    matches = [
        article for article in news_docs
        if ticker in article.get('tickers', '')
    ]
    return matches

In [69]:
# Test exact match
tsla_exact = get_news_by_exact_ticker("TSLA")
print(f"\n--- Exact ticker matches for TSLA: {len(tsla_exact)} articles ---")



--- Exact ticker matches for TSLA: 133 articles ---


In [71]:
# Step 8: Multi-ticker search with progress bar
def search_multiple_tickers(tickers, num_results=20):
    """
    Search for news mentioning any of the given tickers.
    """
    all_results = {}
    for ticker in tqdm(tickers, desc="Searching tickers"):
        results = search_news_by_ticker(ticker, num_results=num_results)
        all_results[ticker] = results
    return all_results

In [72]:
# Example: Get news for MAG7 stocks
print("\n" + "="*70)
print("Searching for MAG7 stocks...")
print("="*70)

mag7_tickers = ["AAPL", "GOOGL", "META", "MSFT", "TSLA", "AMZN", "NVDA"]
mag7_news = search_multiple_tickers(mag7_tickers, num_results=5)

print("\nResults summary:")
for ticker, articles in mag7_news.items():
    print(f"\n{ticker}: {len(articles)} articles found")
    if articles:
        print(f"  Latest: {articles[0]['title'][:80]}...")


Searching for MAG7 stocks...


Searching tickers: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 267.67it/s]


Results summary:

AAPL: 5 articles found
  Latest: Every Apple (AAPL) Investor Should Keep an Eye on This Number...

GOOGL: 5 articles found
  Latest: Allied Announces Third-Quarter Results...

META: 5 articles found
  Latest: Should You Buy Meta Platforms Before Oct. 29?...

MSFT: 5 articles found
  Latest: Microsoft's New OpenAI Deal Could Be Its Smartest AI Move Yet...

TSLA: 5 articles found
  Latest: Tesla Q3 Deliveries Smash Estimates, But Wall Street Wasn't Impressed. What Give...

AMZN: 5 articles found
  Latest: 1 Reason Why Amazon (AMZN) Is the Ultimate Growth Stock to Buy With $1,000 Right...

NVDA: 5 articles found
  Latest: Think You Missed the Boat on Nvidia? Here's the No. 1 Reason It Could Keep Climb...


# Q3: Persistent vs. in-memory storage

## Reco: Persistent Storage (SQLite + Periodic Updates)

✅ Persistent & Accumulative:
  - SQLite stores all historical news
  - Each run adds only new articles (no duplicates)
  - Survives restarts/crashes
  - Can query historical data

  ✅ Efficient Updates:
  - Only fetches news newer than what's in DB
  - Respects API rate limits
  - Fast incremental updates

  ✅ Flexible Querying:
  - SQL queries for complex filters
  - Direct ticker lookups
  - Date-range queries
  - Can export to other tools

  ✅ Scalable:
  - SQLite handles millions of rows
  - Indexed for fast queries
  - Can migrate to PostgreSQL if needed

  ✅ Easy Scheduling:
**Cron job for daily updates (Linux/Mac)**

  0 9 * * * cd /path/to/project && uv run python update_news.py

**Or use Python scheduler**

  import schedule
  schedule.every().day.at("09:00").do(update_and_search_workflow, POLYGON_API_KEY)

  For your use case with regular updates: SQLite + minsearch is the sweet spot! 🎯

In [ ]:
# code:
# TODO: cache function calls in the db too for many tickers??

#   import sqlite3
#   import pandas as pd
#   from datetime import datetime, timezone
#   from pathlib import Path
#   from tqdm import tqdm

#   # Configuration
#   DB_DIR = Path("data")
#   DB_DIR.mkdir(parents=True, exist_ok=True)
#   DB_FILE = DB_DIR / "news_database.db"

#   class NewsDatabase:
#       """Persistent news storage with SQLite."""

#       def __init__(self, db_path=DB_FILE):
#           self.db_path = db_path
#           self.conn = sqlite3.connect(db_path)
#           self._create_tables()

#       def _create_tables(self):
#           """Create news table if it doesn't exist."""
#           self.conn.execute("""
#               CREATE TABLE IF NOT EXISTS news (
#                   id TEXT PRIMARY KEY,
#                   title TEXT,
#                   author TEXT,
#                   published_utc TEXT,
#                   article_url TEXT,
#                   tickers TEXT,
#                   image_url TEXT,
#                   description TEXT,
#                   keywords TEXT,
#                   publisher_name TEXT,
#                   publisher_homepage_url TEXT,
#                   publisher_logo_url TEXT,
#                   publisher_favicon_url TEXT,
#                   amp_url TEXT,
#                   insights TEXT,
#                   created_at TEXT DEFAULT CURRENT_TIMESTAMP
#               )
#           """)

#           # Create index on published_utc for fast date queries
#           self.conn.execute("""
#               CREATE INDEX IF NOT EXISTS idx_published_utc 
#               ON news(published_utc)
#           """)

#           # Create index on tickers for fast ticker searches
#           self.conn.execute("""
#               CREATE INDEX IF NOT EXISTS idx_tickers 
#               ON news(tickers)
#           """)

#           self.conn.commit()

#       def get_latest_news_date(self):
#           """Get the most recent news date in database."""
#           cursor = self.conn.execute("""
#               SELECT MAX(published_utc) FROM news
#           """)
#           result = cursor.fetchone()[0]
#           return result if result else None

#       def add_news(self, news_df):
#           """
#           Add news articles to database (skip duplicates).
          
#           Args:
#               news_df: DataFrame with news articles
          
#           Returns:
#               Number of new articles added
#           """
#           # Preprocess: convert lists to strings
#           news_df = news_df.copy()

#           for col in ['tickers', 'keywords']:
#               if col in news_df.columns:
#                   news_df[col] = news_df[col].apply(
#                       lambda x: ', '.join(x) if isinstance(x, list) else str(x)
#                   )

#           # Flatten nested publisher fields
#           if 'publisher.name' in news_df.columns:
#               news_df['publisher_name'] = news_df['publisher.name']
#               news_df['publisher_homepage_url'] = news_df['publisher.homepage_url']
#               news_df['publisher_logo_url'] = news_df['publisher.logo_url']
#               news_df['publisher_favicon_url'] = news_df['publisher.favicon_url']

#           # Convert insights to JSON string if exists
#           if 'insights' in news_df.columns:
#               news_df['insights'] = news_df['insights'].apply(str)

#           # Select relevant columns
#           cols = [
#               'id', 'title', 'author', 'published_utc', 'article_url',
#               'tickers', 'image_url', 'description', 'keywords',
#               'publisher_name', 'publisher_homepage_url',
#               'publisher_logo_url', 'publisher_favicon_url',
#               'amp_url', 'insights'
#           ]

#           news_to_insert = news_df[[c for c in cols if c in news_df.columns]]

#           # Insert with conflict ignore (skip duplicates)
#           initial_count = self.count_articles()
#           news_to_insert.to_sql('news', self.conn, if_exists='append',
#                                 index=False, method='multi')
#           final_count = self.count_articles()

#           new_articles = final_count - initial_count
#           return new_articles

#       def count_articles(self):
#           """Get total number of articles in database."""
#           cursor = self.conn.execute("SELECT COUNT(*) FROM news")
#           return cursor.fetchone()[0]

#       def get_all_news(self):
#           """Load all news from database as DataFrame."""
#           return pd.read_sql_query("SELECT * FROM news", self.conn)

#       def get_news_by_ticker(self, ticker):
#           """Get news for a specific ticker."""
#           query = """
#               SELECT * FROM news 
#               WHERE tickers LIKE ? 
#               ORDER BY published_utc DESC
#           """
#           return pd.read_sql_query(query, self.conn, params=[f'%{ticker}%'])

#       def get_news_since(self, date):
#           """Get news published after a specific date."""
#           query = """
#               SELECT * FROM news 
#               WHERE published_utc > ? 
#               ORDER BY published_utc DESC
#           """
#           return pd.read_sql_query(query, self.conn, params=[date])

#       def close(self):
#           """Close database connection."""
#           self.conn.close()


#   # Fetch and update workflow
#   def fetch_and_update_news(api_key, db=None):
#       """
#       Fetch new news and add to database.
#       Only fetches news newer than what's already in DB.
      
#       Args:
#           api_key: Polygon.io API key
#           db: NewsDatabase instance (optional)
      
#       Returns:
#           Number of new articles added
#       """
#       if db is None:
#           db = NewsDatabase()

#       # Get latest date in database
#       latest_date = db.get_latest_news_date()

#       if latest_date:
#           print(f"📅 Latest news in DB: {latest_date}")
#           print(f"🔄 Fetching news newer than {latest_date}...")
#           max_date = datetime.now(timezone.utc).isoformat(timespec='milliseconds').replace('+00:00', 'Z')
#       else:
#           print("📭 Database is empty. Fetching all available news...")
#           max_date = datetime.now(timezone.utc).isoformat(timespec='milliseconds').replace('+00:00', 'Z')

#       # Fetch news (your existing function)
#       new_news = get_all_news(api_key=api_key, news_limit=1000, max_date=max_date)

#       if len(new_news) == 0:
#           print("✓ No new articles to add")
#           return 0

#       # Filter out articles older than latest_date
#       if latest_date:
#           new_news = new_news[new_news['published_utc'] > latest_date]

#       if len(new_news) == 0:
#           print("✓ No new articles to add (all were older)")
#           return 0

#       # Add to database
#       print(f"💾 Adding {len(new_news)} new articles to database...")
#       added_count = db.add_news(new_news)

#       print(f"✓ Added {added_count} new articles")
#       print(f"📊 Total articles in database: {db.count_articles()}")

#       return added_count


#   # Build search index from database
#   def build_search_index(db=None):
#       """
#       Build minsearch index from database.
#       Call this after updating database.
#       """
#       if db is None:
#           db = NewsDatabase()

#       print("\n🔍 Building search index...")

#       # Load all news from database
#       all_news_df = db.get_all_news()
#       news_documents = all_news_df.to_dict(orient='records')

#       # Preprocess for minsearch
#       print("⚙️ Preprocessing documents...")
#       for doc in tqdm(news_documents, desc="Processing"):
#           # Ensure strings (already done in DB but just in case)
#           for field in ['title', 'description', 'author', 'tickers', 'keywords']:
#               if doc.get(field) is None:
#                   doc[field] = ''
#               elif not isinstance(doc.get(field), str):
#                   doc[field] = str(doc[field])

#       # Create and fit index
#       from minsearch import Index

#       news_index = Index(
#           text_fields=["title", "description", "keywords", "author", "tickers"],
#           keyword_fields=["published_utc", "publisher_name"]
#       )

#       news_index.fit(news_documents)
#       print(f"✓ Search index built with {len(news_documents)} articles")

#       return news_index, news_documents


#   # Main workflow for regular updates
#   def update_and_search_workflow(api_key):
#       """
#       Complete workflow: fetch new news, update DB, rebuild index.
#       Run this daily/hourly via cron or scheduler.
#       """
#       print("="*70)
#       print("📰 NEWS UPDATE WORKFLOW")
#       print("="*70)

#       # 1. Initialize database
#       db = NewsDatabase()
#       print(f"\n📊 Current database stats:")
#       print(f"   Total articles: {db.count_articles()}")

#       # 2. Fetch and add new news
#       added = fetch_and_update_news(api_key, db)

#       # 3. Rebuild search index (if new articles added)
#       if added > 0 or db.count_articles() > 0:
#           news_index, news_documents = build_search_index(db)
#       else:
#           news_index, news_documents = None, []

#       # 4. Close database
#       db.close()

#       print("\n✓ Update workflow complete!")
#       print("="*70)

#       return news_index, news_documents


#   # Usage examples:

#   # Run once to initialize
#   news_index, news_documents = update_and_search_workflow(POLYGON_API_KEY)

#   # Later - just run again to get updates
#   # news_index, news_documents = update_and_search_workflow(POLYGON_API_KEY)

#   # Search using the index
#   def search_news_by_ticker(ticker, news_index, num_results=10):
#       results = news_index.search(
#           query=ticker,
#           num_results=num_results,
#           boost_dict={
#               'tickers': 5.0,
#               'title': 3.0,
#               'description': 1.5
#           }
#       )
#       return results

#   # Query database directly
#   db = NewsDatabase()
#   tsla_news_df = db.get_news_by_ticker("TSLA")
#   print(f"Found {len(tsla_news_df)} TSLA articles in database")

#   # Get recent news
#   recent_news = db.get_news_since("2025-10-25T00:00:00Z")
#   print(f"Found {len(recent_news)} articles since Oct 25")

#   db.close()

# Part 2: Build a Summary Agent Using Wikipedia Pages`

# Q4: Framework and LLM Provider
Answer: easiest option: 
  1. Framework: OpenAI Agents SDK - chosen for its simplicity and native tool calling
  2. Provider: OpenAI with gpt-4o-mini for cost-effectiveness
  3. Tools: Three financial analysis tools integrated

# Q5: Agent Instructions (fetch and save)

In [73]:
# Imports
from agents import Agent, Runner, function_tool
import requests
from typing import Optional
from requests.exceptions import RequestException
import os
from pathlib import Path

# Tool 1: Fetch Web Page
def fetch_url(url: str) -> Optional[str]:
    """
    Fetch the textual content of a webpage using Jina Reader.
      
    Args:
        url: The target URL to fetch content from (e.g., 'https://example.com')
      
    Returns:
        The text content of the webpage, or None if an error occurred
    """
    if not url or not isinstance(url, str):
        raise ValueError("The 'url' parameter must be a non-empty string.")

    jina_reader_base_url = "https://r.jina.ai/"
    jina_reader_url = jina_reader_base_url + url.lstrip("/")

    try:
        response = requests.get(jina_reader_url, timeout=30)
        response.raise_for_status()
        return response.content.decode("utf-8")
    except RequestException as e:
        print(f"Error fetching URL '{jina_reader_url}': {e}")
        return None
    except UnicodeDecodeError:
        print(f"Error decoding response from '{jina_reader_url}'.")
        return None     

In [74]:
# Tool 2: Save Summary
def save_summary(summary: str, filename: str = "summary.txt") -> dict:
    """
    Save a text summary to a file in the summaries directory.
      
    Args:
        summary: The text content to save
        filename: Name of the file (default: summary.txt)
      
    Returns:
        Dictionary with status and file path
    """
    try:
        # Create summaries directory if it doesn't exist
        summaries_dir = Path("summaries")
        summaries_dir.mkdir(exist_ok=True)

        # Save summary to file
        file_path = summaries_dir / filename

        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(summary)

        return {
            "status": "success",
            "file_path": str(file_path),
            "message": f"Summary saved to {file_path}",
            "summary_length": len(summary)
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"Failed to save summary: {str(e)}"
        }

In [75]:
  # ANSWER: Final Agent Instructions
AGENT_INSTRUCTIONS = """You are a helpful web research assistant.

When a user asks "What is this page about?" followed by a URL:

STEP 1: Use the fetch_url tool to retrieve the webpage content.
  - Pass the complete URL to the tool
  - Wait for the content to be fetched

STEP 2: After receiving the content, read and analyze it carefully.
  - Identify the main topic and key points
  - Create a clear, concise summary (3-5 paragraphs)

STEP 3: Use the save_summary tool to save your summary.
  - Pass your written summary as the 'summary' parameter
  - Use a descriptive filename based on the page topic (e.g., 'capybara_summary.txt')

STEP 4: Confirm to the user that:
  - The page content was fetched successfully
  - Your summary of what the page is about
  - The location where the summary was saved

IMPORTANT:
- Always use both tools in sequence: fetch_url first, then save_summary
- Do not skip the save_summary step
- Provide a helpful summary in your response even after saving
- If fetch_url fails, explain the error and do not attempt to save"""

# Create the agent with proper instructions
web_research_agent = Agent(
    name="Web Research Assistant",
    model="gpt-4o-mini",
    instructions=AGENT_INSTRUCTIONS,
    tools=[
          function_tool(fetch_url),
          function_tool(save_summary)
      ]
)

print("✓ Agent created with instructions")
print(f"✓ Tools available: {[tool.name for tool in web_research_agent.tools]}")

✓ Agent created with instructions
✓ Tools available: ['fetch_url', 'save_summary']


In [76]:
# Test the agent
async def test_web_research(url: str):
    """
    Test the web research agent with a URL.
      
    Args:
        url: The URL to analyze
    """
    query = f"What is this page about? {url}"

    print("\n" + "="*70)
    print(f"Query: {query}")
    print("="*70)

    results = await Runner.run(
        web_research_agent,
        input=query,
        max_turns=15
    )

    print("\n" + "="*70)
    print("Agent Response:")
    print("="*70)
    print(results.final_output)

    return results

In [77]:
# Example usage
print("\n" + "="*70)
print("Testing with Capybara Wikipedia page")
print("="*70)

# Uncomment to run:
results = await test_web_research("https://en.wikipedia.org/wiki/Capybara")

print("\n✓ Agent setup complete!")


Testing with Capybara Wikipedia page

Query: What is this page about? https://en.wikipedia.org/wiki/Capybara

Agent Response:
The page about capybaras on Wikipedia provides a comprehensive overview of this fascinating rodent species. Here’s a summary of the key points:

The **capybara**, or greater capybara (*Hydrochoerus hydrochaeris*), is recognized as the largest living rodent and is native to South America. These creatures thrive in various habitats, including savannas and dense forests, but are particularly drawn to environments near water bodies. Known for their social behavior, capybaras often form large groups, varying from 10 to as many as 100 individuals. Their diet mainly includes grasses and aquatic plants, and they display unique feeding behaviors such as coprophagy, which allows them to extract maximum nutrients from their food.

Capybaras are remarkable swimmers, able to stay submerged for up to five minutes, which helps them evade predators. They primarily graze on gra

# Q6: Search Function with Multiple Document Indexing

In [78]:
from agents import Agent, Runner, function_tool
import requests
from typing import Optional, List, Dict, Any
from requests.exceptions import RequestException
from pathlib import Path
from minsearch import Index
from tqdm import tqdm

# Global storage for indexed documents
indexed_documents = []
search_index = None

# Tool 1: Fetch and Index Web Pages
def fetch_and_index_pages(urls: List[str]) -> Dict[str, Any]:
    """
    Fetch multiple web pages and add them to the search index.
      
    Args:
        urls: List of URLs to fetch and index
      
    Returns:
        Dictionary with indexing results
    """
    global indexed_documents, search_index

    jina_reader_base_url = "https://r.jina.ai/"
    newly_indexed = 0
    failed = []

    print(f"\n📥 Fetching and indexing {len(urls)} pages...")

    for url in tqdm(urls, desc="Indexing pages"):
        try:
            # Fetch content
            jina_url = jina_reader_base_url + url.lstrip("/")
            response = requests.get(jina_url, timeout=30)
            response.raise_for_status()
            content = response.content.decode("utf-8")

            # Create document
            doc = {
                'url': url,
                'content': content,
                'title': url.split('/')[-1].replace('_', ' ')
            }

            indexed_documents.append(doc)
            newly_indexed += 1

        except Exception as e:
            failed.append({'url': url, 'error': str(e)})

    # Rebuild search index
    if indexed_documents:
        search_index = Index(text_fields=['content', 'title', 'url'])
        search_index.fit(indexed_documents)

    return {
        'status': 'success',
        'newly_indexed': newly_indexed,
        'total_documents': len(indexed_documents),
        'failed': failed,
        'message': f"Successfully indexed {newly_indexed} pages. Total documents: {len(indexed_documents)}"
    }



In [79]:
# Tool 2: Search Indexed Documents
def search_documents(query: str, num_results: int = 3) -> Dict[str, Any]:
    """
    Search through indexed documents for relevant information.
      
    Args:
        query: Search query string
        num_results: Number of results to return (default: 3)
      
    Returns:
        Dictionary with search results and the query used
    """
    global search_index, indexed_documents

    if not search_index or not indexed_documents:
        return {
            'status': 'error',
            'message': 'No documents indexed. Please index documents first.',
            'query_used': query,
            'results': []
        }

    try:
        results = search_index.search(
            query=query,
            num_results=num_results,
            boost_dict={'title': 2.0, 'content': 1.0}
        )

        # Format results with excerpts
        formatted_results = []
        for result in results:
            # Get relevant excerpt (first 500 chars containing query terms)
            content = result['content']
            query_lower = query.lower()

            # Find position of query terms in content
            pos = content.lower().find(query_lower)
            if pos == -1:
                # If exact match not found, take first 500 chars
                excerpt = content[:500]
            else:
                # Get context around the query
                start = max(0, pos - 200)
                end = min(len(content), pos + 300)
                excerpt = content[start:end]

            formatted_results.append({
                'title': result['title'],
                'url': result['url'],
                'excerpt': excerpt + '...'
            })

        return {
            'status': 'success',
            'query_used': query,
            'num_results': len(formatted_results),
            'results': formatted_results
        }

    except Exception as e:
        return {
            'status': 'error',
            'message': f"Search failed: {str(e)}",
            'query_used': query,
            'results': []
        }



In [80]:
# Tool 3: Save Summary (from Q5)
def save_summary(summary: str, filename: str = "summary.txt") -> dict:
    """
    Save a text summary to a file in the summaries directory.
      
    Args:
        summary: The text content to save
        filename: Name of the file (default: summary.txt)
      
    Returns:
        Dictionary with status and file path
    """
    try:
        summaries_dir = Path("summaries")
        summaries_dir.mkdir(exist_ok=True)
        file_path = summaries_dir / filename

        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(summary)

        return {
            "status": "success",
            "file_path": str(file_path),
            "message": f"Summary saved to {file_path}"
        }
    except Exception as e:
        return {
            "status": "error",
            "message": f"Failed to save summary: {str(e)}"
        }

In [81]:
# ANSWER: Enhanced Agent Instructions with Search
SEARCH_AGENT_INSTRUCTIONS = """You are a research assistant specializing in information retrieval and synthesis.

  Your capabilities:
  1. fetch_and_index_pages: Index web pages into a searchable database
  2. search_documents: Search indexed documents for specific information
  3. save_summary: Save your findings to a file

  WORKFLOW FOR INDEXING MULTIPLE PAGES:
  When asked to index multiple pages, use fetch_and_index_pages with ALL URLs in a single call.
  Example: fetch_and_index_pages(urls=['url1', 'url2', 'url3', ...])

  WORKFLOW FOR ANSWERING QUESTIONS:
  When asked a question about indexed content:

  STEP 1: Formulate 2-3 specific search queries related to the question.
  - Break down the question into key concepts
  - Create targeted search queries for each concept
  - Example question: "What are threats to capybara populations?"
    Search queries could be:
    * "threats capybara population"
    * "capybara predators dangers"
    * "capybara conservation habitat loss"

  STEP 2: Use search_documents for EACH search query.
  - Call search_documents multiple times with different queries
  - Collect information from all search results

  STEP 3: Synthesize findings from all searches.
  - Combine information from multiple search results
  - Identify common themes and specific details
  - Organize findings logically

  STEP 4: Provide comprehensive answer to the user.
  - List the search queries you used
  - Summarize key findings
  - Include specific details from the documents

  IMPORTANT:
  - ALWAYS use search_documents when asked a question about indexed content
  - Use multiple different search queries to gather comprehensive information
  - Show your search queries in the response so users know what you searched for
  - Synthesize information from all results into a coherent answer
  - If results are insufficient, try alternative search queries"""